In [1]:
from numba import cuda, vectorize, float32
from scipy import sparse
import random
import math
import string

import time
import numpy as np

In [2]:
def onehot_string(x, max_length):
    
    if x!=x:
        return None
    
    #Pad to max length
    while len(x)<max_length:
        x+=' '
        
    sparse_matrix = sparse.hstack([sparse.csr_matrix(
        [0 if char_dict[current_char]!=i else 1 for i in range(len(char_dict))]) for current_char in x])
    
    return sparse_matrix


In [6]:
num_of_chars = 208

In [7]:
a = np.array([[random.randrange(0, 2) for i in range(num_of_chars)] for i in range(200)])
b = np.array([[random.randrange(0, 2) for i in range(num_of_chars)] for i in range(400)])

In [8]:
b_t = np.transpose(b)

In [9]:
start_time = time.time()
c_true = np.matmul(a, b_t)
end_time = time.time()

In [10]:
end_time-start_time

0.029898405075073242

In [11]:
c_true.shape

(200, 400)

In [12]:
@cuda.jit
def test(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
            
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += abs(A[i, k] - B[k, j])
        C[i, j] = tmp

In [13]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

TPB_x=2
TPB_y=4
        
@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [14]:
a = np.array([[random.randrange(0, 2) for i in range(num_of_chars)] for i in range(200)])
b = np.array([[random.randrange(0, 2) for i in range(num_of_chars)] for i in range(400)])

In [15]:
#c = np.zeros(shape=(a.shape[0], b.shape[0]))

In [16]:
a.shape, b.shape

((200, 208), (400, 208))

In [17]:
TPB = 256 #<---threads per block

blockspergrid_x = int(math.ceil(a.shape[0] / TPB))
blockspergrid_y = int(math.ceil(b.shape[0] / TPB))

blockspergrid = (blockspergrid_x, blockspergrid_y)

TPB, blockspergrid

(256, (1, 2))

In [18]:
threadsperblock

NameError: name 'threadsperblock' is not defined

In [77]:
c = np.zeros(shape=(a.shape[0], b.shape[0]))
start_time = time.time()
#fast_matmul[[TPB, TPB], blockspergrid](a, b, c)
matmul[threadsperblock, blockspergrid](a, b, c)
end_time = time.time()

CudaAPIError: [700] Call to cuMemAlloc results in CUDA_ERROR_LAUNCH_FAILED

In [52]:
end_time-start_time

0.06086397171020508

In [34]:
c

array([[100., 100., 100., ..., 100., 100., 100.],
       [100., 100., 100., ..., 100., 100., 100.],
       [100., 100., 100., ..., 100., 100., 100.],
       ...,
       [100., 100., 100., ..., 100., 100., 100.],
       [100., 100., 100., ..., 100., 100., 100.],
       [100., 100., 100., ..., 100., 100., 100.]])

In [1]:
blocks = 100

threads_x = a.shape[0]//blocks
threads_y = b.shape[0]//blocks

threads_x, threads_y
#c = np.zeros(shape=(a.shape[0], b.shape[0]))
#start_time = time.time()
#matmul[[blocks, blocks], [threads_x, threads_y]](a, b_t, c)
#end_time = time.time()

In [150]:
def test_manual(a, b, i):
    tmp = 0
    for j in range(b.shape[0]):
        tmp+=abs(a[i]-b[j])
        
    return tmp